In [ ]:
def train_nn(data, save_path):
    """
    Train a neural network to predict buy/sell signals based on market data.

    Args:
        data (pd.DataFrame): Historical market data (including 'Adj Close').
        save_path (str): Path to save the trained neural network model.

    Returns:
        None
    """
    data = data.copy()

    # Prepare features and labels
    data['Return'] = data['Adj Close'].pct_change()
    data['Target'] = np.where(data['Return'].shift(-1) > 0, 1, 0)
    data.dropna(inplace=True)

    features = data[['Return']].values
    labels = data['Target'].values

    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Build a simple neural network
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_train.shape[1],)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=32)

    # Save the trained model
    model.save(save_path)


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd


def optimize_nn(data, param_grid, save_path, epochs=50):
    """
    Optimize a neural network by tuning hyperparameters.

    Args:
        data (pd.DataFrame): Historical market data (including 'Adj Close').
        param_grid (dict): Grid of hyperparameters to test.
        save_path (str): Path to save the best neural network model.
        epochs (int): Number of training epochs.

    Returns:
        dict: Best hyperparameters and their performance.
    """
    data = data.copy()

    # Prepare features and labels
    data['Return'] = data['Adj Close'].pct_change()
    data['Target'] = np.where(data['Return'].shift(-1) > 0, 1, 0)
    data.dropna(inplace=True)

    features = data[['Return']].values
    labels = data['Target'].values

    # Standardize features
    scaler = StandardScaler()
    features = scaler.fit_transform(features)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # S
